In [1]:
# ignore and not displaying all warnings generated by any code
import warnings
warnings.filterwarnings('ignore')

# jijmodeling_transpiler tutorial

jijmodeling transpiler is a tool to transpile mathmatical models between JijModeling and other modeling platforms. Currently it supports QUBO (Quadratic Unconstrained Binary Optimization) for QUBO solvers and to Python-MIP for MIP solvers.

To solve optimization problem using JijModelingTranspiler, we follow the following steps: 
- Define problem with mathatical model using JijModeling library, and set instance data 
- Compile and transpile a model using JijModelingTranspiler library
- Solve a problem and find a soltion using OpenJij library

## Installation
jijmodeling-transpiler and other necessary Jij libraries / modules can be install using pip

In [2]:
# !pip install jijmodeling
# !pip install jijmodeling-transpiler
# !pip install openjij

then, you can import them to your code with general import command with conventional short name

In [3]:
import jijmodeling as jm
import jijmodeling.transpiler as jmt
import openjij as oj

## Knapsack problem

The knapsack problem is well known combinatorial optimization NP-hard problem. In general case a set of item is given and each item has different value and weight. The goal is to find the optimal set of item that total weight is less than the given limit and the total value is as large as possible.


For this tutorial, we use the following example: (https://www.documentation.jijzept.com/docs/tutorial/knapsack/).
Price and weight of each treasure is given below and capacity of knapsack is __2kg (2000g)__ which is a constrain for this example. Our goal is to find a optimal combination of treasure with total weight being less than 2kg and has as higer total price as possible.

| | Treasure A | Treasure B | Treasure C | Treasure D | Treasure E | Treasure F | 
| :- |:- |:- |:- |:- |:- |:- |
|Price ($)|5000|7000|2000|1000|4000|3000|
|Weight (g)|800|1000|600|400|500|300|

First, we consider generalization of above problem. Let $ N $ be a total number of treasures. We define a list of price $ v $ and weight $ w $ as follow: 
$$ v = \{ v_0, v_1, v_2, ...., v_{N-1} \} $$
$$ w = \{ w_0, w_1, w_2, ...., w_{N-1} \} $$
We also define a binary variable $ x \in \{ 0, 1 \}$, which represent a selection of treasures.
- 1 : if we put a treasure in the knapsack
- 0 : otherwise 

Finally, we let $ W $ be the capacity of the knapsack.

Now, let us express this problem as a mathmatical model. 
<p> First, we like to maximizes the total price of treasures in the knapsack </p>
\begin{equation}
\max \sum_{i=0}^{N-1}v_i x_i
\end{equation}
$$ x \in \{ 0, 1 \} $$
<p>We need to take into account the constrain which is the capacity of a knapsack $ W $ in this case </p>
\begin{equation}
s.t. \sum_{i=0}^{N-1} w_i x_i \leq W
\end{equation}
$$ x \in \{ 0, 1 \} $$

We can implement the defined mathematical model above using JijModeling as follow (please refer to https://www.documentation.jijzept.com/docs/tutorial/knapsack/ for a detail)

## Define mathmartical model for the problem

In [4]:
# define variables
v = jm.Placeholder('v', dim=1)
N = v.shape[0].set_latex('N')
w = jm.Placeholder('w', shape=(N))
W = jm.Placeholder('W')
x = jm.Binary('x', shape=(N))
i = jm.Element('i', (0, N))

In [5]:
# set problem
problem = jm.Problem('Knapsack')
# set objective function
obj = - jm.Sum(i, v[i]*x[i])
problem += obj

In [6]:
# set total weight constarint 
const = jm.Sum(i, w[i]*x[i])
problem += jm.Constraint('weight', const<=W)

Let's print problem we implemented with JijModeling below

In [7]:
problem

Now let's prepare data for this example as define on the table above.

In [8]:
#create a list for price (v) and weight (w)
price = [5000, 7000, 2000, 1000, 4000, 3000]
weight = [800, 1000, 600, 400, 500, 300]
#set the capacity (constrain)
capacity = 2000

data = {'v': price, 'w':weight, 'W':capacity}

Alternativly, you can create those lists with random values.

In [9]:
# import numpy as np
# # create a list for price (v) and weight (w)
# price = np.random.randint(5,30,5)
# weight = price + np.random.randint(-2,20,5)
# # set the capacity (constrain)
# capacity = 100
# data = {'v': price, 'w':weight, 'W':capacity}   

Now, we have the following set of data for the problem.

In [10]:
data

{'v': [5000, 7000, 2000, 1000, 4000, 3000],
 'w': [800, 1000, 600, 400, 500, 300],
 'W': 2000}

## Compile and transpile a model 

From here, we compile and transpile the problem into a ceratin model using JijModelingTranspiler (https://www.documentation.jijzept.com/docs/jijmodelingtranspiler/)

First we compile the model using *compiled_model()* . We pass JijModel we define above and instant data we set.

In [11]:
compiled_model = jmt.core.compile_model(problem, data, {})

Now, we transpile this compiled model into a Quadratic Unconstraint Binary Optimization (QUBO) model using *pubo.transpile_to_pubo()*

In general, given a combinational optimization problem with objective function
$ \min \; f(\vec{x})$ with a constraint $ h(\vec{x}) = c$, QUBO can be form as 
$$ \min \; f(\vec{x}) + \mu (h(\vec{x}) - c)^2 $$ where $\mu$ is a multipliers variable and $\mu \geq 0$

Specifically, this call penalty method and let us define $f(\vec{x})$ and $h(\vec{x})$ as $f(\vec{x}) = \vec{x} \bar{\bar{Q}} \vec{x} \;\; , h(\vec{x}) = \vec{a} \cdot \vec{x} = c$ then we have

$$ \min \; \vec{x} \bar{\bar{Q}} \vec{x}  + \mu (\vec{a} \cdot \vec{x} - c)^2 $$
where $\bar{\bar{Q}}$ is a sparse matrix.

In [12]:
# Quadratic Unconstraint Binary Optimization (QUBO) model
pubo_builder = jmt.core.pubo.transpile_to_pubo(compiled_model=compiled_model)

In [13]:
#set multiplier "onehot" to 1.
qubo,const = pubo_builder.get_qubo_dict(multipliers = {'onehot': 1.0})

Multipliers (ex. $\mu$ on above equation) set an uniformal weight for each penalties. In default it is set to 1. In this example, we set it to 1 since there is no need to manipulate penalty perameters. Deciding optimal permeter require other algorithms. If your result is not good such that solution violate constrain, it is most likely because of multipliers parameters. So adjusting multipliers parameter will leads to optimal solution. 

In [14]:
const

2.0

In [15]:
qubo

{(5, 5): -1.2385714285714284,
 (4, 4): -1.8214285714285712,
 (3, 3): -1.1828571428571428,
 (2, 2): -1.7257142857142855,
 (1, 1): -3.0,
 (0, 0): -2.474285714285714,
 (4, 5): 0.3,
 (3, 5): 0.24,
 (2, 5): 0.36,
 (1, 5): 0.6,
 (0, 5): 0.48,
 (3, 4): 0.4,
 (2, 4): 0.6,
 (1, 4): 1.0,
 (0, 4): 0.8,
 (2, 3): 0.48,
 (1, 3): 0.8,
 (0, 3): 0.6400000000000001,
 (1, 2): 1.2,
 (0, 2): 0.96,
 (0, 1): 1.6}

*qubo* variable is a $Q$ matrix. It is a sparse matrix and above data shows matrix index and its value where it is nonzero. 

Alternativly, you can use MIPBuilder class to compile to Python-MIP if that is more suitable for you. 

## Solving the problem

In this step, we solve the problem we define and decode a result into a JijModeling sample set.

In [16]:
#set sampler 
sampler = oj.SASampler()

#solve the problem 
result = sampler.sample_qubo(qubo)


This is a result of computation before decoding.

In [17]:
result

Response(rec.array([([0, 1, 0, 0, 1, 1], -4.16, 1)],
          dtype=[('sample', 'i1', (6,)), ('energy', '<f8'), ('num_occurrences', '<i8')]), Variables([0, 1, 2, 3, 4, 5]), {'system': [], 'sampling_time': 8216.055000000111, 'execution_time': 7876.165999999962, 'list_exec_times': array([7876.166]), 'schedule': {'beta_max': 123.98535116121748, 'beta_min': 0.46209812037329684, 'num_sweeps': 1000}}, 'BINARY')

In the response above, a rec.array corresponds to the solution for this example problem. 
More in detail, those are $ x $ values which is a binary variable as we defined previously. 

Now, we can decode this result from OpenJij into a JijModeling sample set using *pubo.decode_from_openjij*. It takes three parameters. Result from OpenJij, pubo_builder which we define above, and compiled_model which is compiled model for this problem. 

In [18]:
#decode a result to JijModeling sample set
sampleset = jmt.core.pubo.decode_from_openjij(result,pubo_builder,compiled_model)

This is a result of computation after decoding from OpenJij response to JijModeling sample set.

In [19]:
sampleset

SampleSet(record=Record(solution={'x': [(([1, 4, 5],), [1, 1, 1], (6,))]}, num_occurrences=[1]), evaluation=Evaluation(energy=array([-4.16]), objective=[-14000.0], constraint_violations={'weight': [0.0]}, penalty={}), measuring_time=MeasuringTime(solve=SolvingTime(preprocess=None, solve=None, postprocess=None), system=SystemTime(post_problem_and_instance_data=None, request_queue=None, fetch_problem_and_instance_data=None, fetch_result=None, deserialize_solution=None), total=None))

In the sampleset, first list of solution dictionaly represent a index of each treasure in optimal set.

In [20]:
#extract a solution list from sampleset.
opt = list(sampleset.record.solution.values())
opt = opt[0][0][0][0]
opt

[1, 4, 5]

## Visualising the solution

In [21]:
from itertools import compress

Therefore, we can conclude that a optimal combination of tresures is Treasure B, E, and F.

In [22]:
result_price = [data['v'][i] for i in opt]
result_weight = [data['w'][i] for i in opt]

In [23]:
print('Price of chosen items: ', result_price)
print('Weight of chosen items: ', result_weight)
print('Total price: ', sum(result_price))
print('Total weight: ', sum(result_weight))
print('Constrain', data['W'])

Price of chosen items:  [7000, 4000, 3000]
Weight of chosen items:  [1000, 500, 300]
Total price:  14000
Total weight:  1800
Constrain 2000


Total weight of optimal set of trasures is indeed less than the constrain $W$